In [19]:
import numpy as np
import networkx as nx
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 6
subSize=10000

In [20]:
def getDegrees(G):
    # sum of in and out degrees by default. use in_degree for specific
    return list(nx.degree(G))

def compareDirect(a,b,size=10,write=""):        
    print("Calculated :\t",a[:size],"...")
    print("")
    print("Expected :\t",b[:size],"...")    
    print("")
#     diffs=[ abs(a-b) for a,b in zip(a,b)]
#     print("Absolute Diffs: ",diffs[:size],"..." if size<len(a) else "")
#     print("")
#     print("Sum of absolute diffs: ",sum(diffs))
#     ratios=[ a/b if b else '0' for a,b in zip(a,b)]
#     print("Ratios: ",ratios)
    if(write!=""):
        a=sorted(a,reverse=True)
        b=sorted(b,reverse=True)
        pd.DataFrame(a).to_csv(write+"_code.csv",header=False,index=None)
        pd.DataFrame(b).to_csv(write+"_tool.csv",header=False,index=None)
        
    print("\n Spearman rank correlation: ")
    print(stats.spearmanr(a,b))
    
    
    
def compare(a,b,name,size=10):
    a=sorted(a,key=lambda x:x[1],reverse=True)
    b=sorted(b,key=lambda x:x[1],reverse=True)
    
    pd.DataFrame(a).to_csv(name+"_code.csv",header=False,index=None)
    pd.DataFrame(b).to_csv(name+"_tool.csv",header=False,index=None)
    
    a=[x[1] for x in a]
    b=[x[1] for x in b]
    compareDirect(a,b,size)

In [21]:
diG=nx.read_weighted_edgelist("higgs-mention_network.edgelist",create_using=nx.DiGraph())#read_edgelist("com-dblp.ungraph.txt")
N=diG.number_of_nodes()
print("Originally: "+str(N)+" nodes")

degrees=sorted(getDegrees(diG),key=lambda x:x[1])
subG=diG.subgraph([x[0] for x in degrees[N-subSize:]])
# subG=diG.subgraph(np.random.choice(list(subG.nodes),size=subSize, replace=False))
subGdegrees=getDegrees(subG)
subN=subG.number_of_nodes()
print("Taken: "+str(subN)+" random nodes from "+str(subSize)+" highest degree nodes")

Originally: 116408 nodes
Taken: 10000 random nodes from 10000 highest degree nodes


In [22]:
G=subG
if(subN<200):    
    pos=dict.fromkeys(G)
    for d in pos:
        pos[d]=(int(20*np.random.rand()),int(20*np.random.rand())) if d!='3998' else (0,0)
    nx.draw_networkx(G,pos)
    labels = nx.get_edge_attributes(G,'weight')
#     nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
    plt.show()

In [23]:
"""
Degree Centrality = degree(n) = In undirected graph it is no of edges 
"""
subNminus=subN-1
degreeCentrality=[(x[0],x[1]/subNminus) for x in subGdegrees]
compare(degreeCentrality,list(nx.degree_centrality(subG).items()),name="degree_centrality")

Calculated :	 [0.2075207520752075, 0.0613061306130613, 0.06050605060506051, 0.045404540454045406, 0.038303830383038306, 0.0338033803380338, 0.0319031903190319, 0.022602260226022602, 0.021602160216021602, 0.020602060206020602] ...

Expected :	 [0.20752075207520754, 0.06130613061306131, 0.06050605060506051, 0.045404540454045406, 0.038303830383038306, 0.0338033803380338, 0.0319031903190319, 0.022602260226022602, 0.021602160216021602, 0.020602060206020602] ...


 Spearman rank correlation: 
SpearmanrResult(correlation=0.9999999999999999, pvalue=0.0)


In [ ]:
# Get all pairs shortest path lengths
all_spls=nx.floyd_warshall(subG) #floyd_warshall gives inf too

In [7]:
"""
Closeness centrality -  is the average length of the shortest path between the node and all other nodes in the graph i.e.
in the connected part of graph containing the node. If the graph is not completely connected, 
this algorithm computes the closeness centrality for each connected part separately.
>> Normalized again with a factor n-1/N-1
"""

closeness=[]
for n,lengths in all_spls.items():
    component=[x for x in lengths.values()  if x!=np.inf ]
    sumDists=sum(component)
    lminus=len(component)-1
    n_fac=lminus/subNminus
    closeness.append( (n,(lminus / sumDists * n_fac) if(sumDists!=0) else 0.0) )
    
#From the tool
ideal=nx.closeness_centrality(subG,distance='weight') #consider the weights

compare(closeness,list(ideal.items()),name="closeness")

Calculated :	 [0.16859957600698341, 0.15972591411187903, 0.15007215007215008, 0.14078933666562535, 0.1344956413449564, 0.11722373954133612, 0.10268094478620794, 0.09685507557847983, 0.09651283149516367, 0.09583554846712743] ...

Expected :	 [0.16859957600698341, 0.15972591411187903, 0.15007215007215008, 0.14078933666562535, 0.1344956413449564, 0.11722373954133612, 0.10268094478620794, 0.09685507557847983, 0.09651283149516367, 0.09583554846712743] ...


 Spearman rank correlation: 
SpearmanrResult(correlation=1.0, pvalue=0.0)


In [8]:
"""
Betweenness centrality computed as follows:
1. For each pair of vertices (s,t), compute the shortest paths between them. <-- spaths
2. For each pair of vertices (s,t), determine the fraction which pass through the vertex v. <-- iterating on paths
3. Sum this fraction over all pairs of vertices (s,t). <-- sum while iterating

Betweenness(v) = no of SPs passing thru v/Total no of SPs
"""
from itertools import count
from heapq import heappush, heappop

def accumulate_from_S(betweenness, S, P, sigma, s):
    delta = dict.fromkeys(S, 0)
    while S:
        w = S.pop()
        coeff = (1.0 + delta[w]) / sigma[w]
        for v in P[w]:
            delta[v] += sigma[v] * coeff # count paths
        if w != s:
            betweenness[w] += delta[w]
    return betweenness

def single_source_dijkstra_multi_sps(G, s, weight='weight'):
    S = []
    P = {}
    for v in G:
        P[v] = []
    sigma = dict.fromkeys(G, 0.0)    # sigma[v]=0 for v in G
    D = {}
    sigma[s] = 1.0
    push = heappush
    pop = heappop
    seen = {s: 0}
    c = count()
    Q = []   # use Q as heap with (distance,node id) tuples
    push(Q, (0, next(c), s, s))
    while Q:
        (dist, _, pred, v) = pop(Q)
        if v in D:
            continue  # already searched this node.
        sigma[v] += sigma[pred]  # count paths
        S.append(v)
        D[v] = dist
        for w, edgedata in G[v].items():
            vw_dist = dist + edgedata.get(weight, 1)
            if w not in D and (w not in seen or vw_dist < seen[w]):
                seen[w] = vw_dist
                push(Q, (vw_dist, next(c), v, w))
                sigma[w] = 0.0
                P[w] = [v]
            
            # """ Multiple paths handled here"""
            elif vw_dist == seen[w]:  # handle equal paths
                sigma[w] += sigma[v]
                P[w].append(v)
                
    return S, P, sigma
                
betweenness = dict.fromkeys(subG, 0.0)  
for s in subG:
    S, P, sigma = single_source_dijkstra_multi_sps(subG, s, 'weight')
    betweenness = accumulate_from_S(betweenness,S,P,sigma,s)

compare(list(betweenness.items()),list(nx.betweenness_centrality(subG,normalized=False,weight='weight').items()),name="betweenness")

Calculated :	 [1429.411904761905, 1336.4452380952382, 775.2595238095241, 738.7928571428573, 727.4785714285714, 506.3333333333333, 356.0, 295.0, 175.1547619047619, 164.82142857142858] ...

Expected :	 [1429.411904761905, 1336.4452380952382, 775.2595238095241, 738.7928571428573, 727.4785714285714, 506.3333333333333, 356.0, 295.0, 175.1547619047619, 164.82142857142858] ...


 Spearman rank correlation: 
SpearmanrResult(correlation=1.0, pvalue=0.0)


In [9]:
# G=nx.path_graph(4)
# eigenvector_centrailty=power_iteration(G,num_simulations=30)

# compareDirect(eigenvector_centrailty,list(nx.eigenvector_centrality(G,weight='weight').values()))#,size=subSize)

In [12]:
"""
Eignevector Centrality
Concept: connections to high-scoring nodes contribute more to the score of the node in question
         than equal connections to low-scoring nodes.
The Eigenvector corresponding to largest eigenvalue gives the desired centrality
For this we use the power method
It is an eigenvalue algorithm: given a diagonalizable matrix A, the algorithm will give greatest(in absolute value) eigenvalue of A,
and a nonzero vector v, the corresponding eigenvector. The algorithm is also known as the Von Mises iteration.[1]

Power iteration is a very simple algorithm, but it may converge slowly. It is suitable for large sparse matrices
"""


def power_iteration(G, num_simulations):
    A=nx.to_numpy_matrix(G);
    A=np.array(A)
    EPSILON = np.longdouble(1e-9)
    b_k = np.random.rand(A.shape[1])
    for _ in range(num_simulations):
#     while 1:
        # calculate the matrix-by-vector product Ab
        b_k1 = np.dot(A, b_k)
        # calculate the norm
        b_k1_norm = np.linalg.norm(b_k1)        
        b_k_next = b_k1 / b_k1_norm
        if(np.sum(abs(b_k_next-b_k))<EPSILON):#*len(b_k)
            break
        b_k=b_k_next
        num_simulations-=1;
    print("left iters: ",num_simulations)
    return b_k_next

eigenvector_centrality=power_iteration(subG,num_simulations=30)

# compare(list(zip(G.nodes,eigenvector_centrality)),list(nx.eigenvector_centrality(subG,weight='weight').items()),name="eigenvector")#,size=subSize)

left iters:  3


array([1.84293051e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       5.33102366e-03, 4.01737497e-01, 7.62496634e-03, 1.31617417e-02,
       4.31021381e-37, 0.00000000e+00, 3.67327233e-37, 0.00000000e+00,
       6.55088767e-02, 5.56330509e-46, 0.00000000e+00, 4.18581705e-27,
       5.37941867e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.87550728e-04,
       7.70987294e-05, 2.37672127e-03, 1.91912997e-32, 0.00000000e+00,
       1.85133947e-03, 1.14805234e-02, 7.06649887e-46, 1.62511388e-04,
       4.59259393e-04, 1.60381237e-45, 0.00000000e+00, 3.54012721e-03,
       3.50221753e-02, 0.00000000e+00, 0.00000000e+00, 2.94930033e-04,
       7.71179373e-02, 1.77846514e-03, 0.00000000e+00, 4.23287786e-46,
       1.12124306e-02, 0.00000000e+00, 0.00000000e+00, 1.28204152e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.91313676e-03,
       0.00000000e+00, 1.00748261e-01, 0.00000000e+00, 4.50239976e-37,
      

In [18]:
y=np.array([1.84293051e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       5.33102366e-03, 4.01737497e-01, 7.62496634e-03, 1.31617417e-02,
       4.31021381e-37, 0.00000000e+00, 3.67327233e-37, 0.00000000e+00,
       6.55088767e-02, 5.56330509e-46, 0.00000000e+00, 4.18581705e-27,
       5.37941867e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.87550728e-04,
       7.70987294e-05, 2.37672127e-03, 1.91912997e-32, 0.00000000e+00,
       1.85133947e-03, 1.14805234e-02, 7.06649887e-46, 1.62511388e-04,
       4.59259393e-04, 1.60381237e-45, 0.00000000e+00, 3.54012721e-03,
       3.50221753e-02, 0.00000000e+00, 0.00000000e+00, 2.94930033e-04,
       7.71179373e-02, 1.77846514e-03, 0.00000000e+00, 4.23287786e-46,
       1.12124306e-02, 0.00000000e+00, 0.00000000e+00, 1.28204152e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.91313676e-03,
       0.00000000e+00, 1.00748261e-01, 0.00000000e+00, 4.50239976e-37,
       0.00000000e+00, 2.85398341e-03, 3.27684593e-02, 0.00000000e+00,
       1.67763480e-02, 5.16686666e-46, 1.59621082e-02, 2.43698504e-03,
       1.82430064e-02, 1.76968483e-03, 6.17823706e-03, 2.58580940e-03,
       3.21299970e-27, 1.72313532e-03, 0.00000000e+00, 1.09488839e-02,
       0.00000000e+00, 0.00000000e+00, 4.58267876e-02, 7.41961580e-04,
       1.50968556e-04, 1.04908160e-37, 1.72313532e-03, 0.00000000e+00,
       1.63443971e-45, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.55303572e-45, 2.97739395e-03, 0.00000000e+00, 1.74597820e-02,
       0.00000000e+00, 0.00000000e+00, 1.64593898e-45, 8.99111838e-01,
       4.88184751e-26, 1.83663617e-37, 0.00000000e+00, 2.53112491e-15,
       3.46204418e-04, 0.00000000e+00, 1.61346580e-02, 2.10348791e-03])

x=np.array([1.84293050e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       5.33102364e-03, 4.01737494e-01, 7.62496630e-03, 1.31617682e-02,
       1.01287748e-30, 0.00000000e+00, 3.54497530e-31, 0.00000000e+00,
       6.55088762e-02, 5.00978995e-38, 0.00000000e+00, 2.20268647e-22,
       5.37941904e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.87551965e-04,
       7.70987289e-05, 2.37672873e-03, 2.24316495e-26, 0.00000000e+00,
       1.85133946e-03, 1.14805234e-02, 8.82490582e-38, 1.62511443e-04,
       4.59259390e-04, 1.73757348e-37, 0.00000000e+00, 3.54012872e-03,
       3.50221769e-02, 0.00000000e+00, 0.00000000e+00, 2.94930031e-04,
       7.71179367e-02, 1.77846513e-03, 0.00000000e+00, 1.63533065e-37,
       1.12124305e-02, 0.00000000e+00, 0.00000000e+00, 1.28204159e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.91313677e-03,
       0.00000000e+00, 1.00748261e-01, 0.00000000e+00, 1.08247742e-30,
       0.00000000e+00, 2.85398340e-03, 3.27684723e-02, 0.00000000e+00,
       1.67763480e-02, 1.77694648e-37, 1.59621089e-02, 2.43698510e-03,
       1.82430064e-02, 1.76968482e-03, 6.17823803e-03, 2.58581157e-03,
       1.69076466e-22, 1.72313530e-03, 0.00000000e+00, 1.09488841e-02,
       0.00000000e+00, 0.00000000e+00, 4.58267873e-02, 7.41961759e-04,
       1.50968560e-04, 3.01750408e-31, 1.72313530e-03, 0.00000000e+00,
       1.98422642e-37, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.67792781e-37, 2.97739694e-03, 0.00000000e+00, 1.74597819e-02,
       0.00000000e+00, 0.00000000e+00, 9.32287397e-38, 8.99111839e-01,
       1.08085553e-21, 1.77248765e-31, 0.00000000e+00, 8.26117523e-13,
       3.46205882e-04, 0.00000000e+00, 1.61346580e-02, 2.10348802e-03])

print(stats.spearmanr(x,y))


SpearmanrResult(correlation=0.9995555837724588, pvalue=2.4708958021469265e-151)


In [26]:
"""
Clustering Coefficient 
The global clustering coefficient is the number of closed triplets (or 3 x triangles) 
over the total number of triplets (both open and closed). 

The local clustering coefficient is the proportion of links between the vertices 
within its neighbourhood divided by the number of links that could possibly exist between them.

Average clustering coefficient is mean of local clusterings
"""
unsubG=nx.to_undirected(subG)
clustering_coeffs={}
for n in unsubG:
    nghs = [e[1] for e in unsubG.edges(n)]
    d=len(nghs)
    e=subG.subgraph(nghs).number_of_edges()    
    # for directed..
    clustering_coeffs[n]= 1*e/(d*(d-1)) if d>1 else 0.0;

compare(list(clustering_coeffs.items()),list(nx.clustering(unsubG).items()),name="clustering")#,size=subSize)

Calculated :	 [2.0, 2.0, 2.0, 2.0, 1.5, 1.5, 1.1666666666666667, 1.1666666666666667, 1.0, 1.0] ...

Expected :	 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8] ...


 Spearman rank correlation: 
SpearmanrResult(correlation=0.956499007414669, pvalue=3.112380269088914e-54)


In [16]:
compareDirect(list(clustering_coeffs.values()),list(nx.clustering(unsubG).values()),write="clustering")

Calculated :	 [0.0, 0.6666666666666666, 0.7333333333333333, 0.3333333333333333, 0.31868131868131866, 2.0, 0.0, 0.6333333333333333, 2.0, 0.25] ...

Expected :	 [0, 0.6666666666666666, 1.0, 0.4175824175824176, 0.358974358974359, 0, 0, 0.5, 1.0, 0.1] ...

Absolute Diffs:  [0.0, 0.0, 0.2666666666666667, 0.08424908424908428, 0.040293040293040316, 2.0, 0.0, 0.1333333333333333, 1.0, 0.15] ...

Sum of absolute diffs:  17.067219912395103

 Spearman rank correlation: 
SpearmanrResult(correlation=0.956499007414669, pvalue=3.112380269088914e-54)


In [63]:

# N=3
# G=nx.grid_2d_graph(N,N)
# pos = dict( (n, n) for n in G.nodes() )
# labels = dict( ((i, j), i + (N-1-j) * N ) for i, j in G.nodes() )
# nx.relabel_nodes(G,labels,False)
# inds=labels.keys()
# vals=labels.values()
# inds=sorted(inds)
# vals=sorted(vals)
# pos2=dict(zip(vals,inds))
# nx.draw_networkx(G, pos=pos2, with_labels=False, node_size = 30)
# plt.show()